# Manejo de datos scrapeados de Sklearn user guide

Estudio de los datos scrapeados de la guia de usuario de Scikit-learn [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Cargo los datos de un archivo pickle previamente descargado por un script de scrapping en Python.

In [1]:
import pickle
import numpy as np

with open('sklearn_guide.plk','rb') as rick:
    df_guide= pickle.load(rick)
df_guide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   level0   649 non-null    object
 1   level1   649 non-null    object
 2   content  649 non-null    object
 3   level2   592 non-null    object
 4   level3   344 non-null    object
 5   level4   58 non-null     object
dtypes: object(6)
memory usage: 30.5+ KB


In [2]:
df_guide.head()

,level0,level1,content,level2,level3,level4
0,Supervised learning,Neural network models (supervised)¶,The following are a set of methods intended fo...,NaN,NaN,NaN
1,Supervised learning,Linear Models¶,It is possible to constrain all the coefficien...,Ordinary Least Squares¶,Non-Negative Least Squares¶,NaN
2,Supervised learning,Linear Models¶,The least squares solution is computed using t...,Ordinary Least Squares¶,Ordinary Least Squares Complexity¶,NaN
3,Supervised learning,Linear Models¶,LinearRegression fits a linear model with coef...,Ordinary Least Squares¶,NaN,NaN
4,Supervised learning,Linear Models¶,Ridge regression addresses some of the problem...,Ridge regression and classification¶,Regression¶,NaN


## Pretratamiento con spacy

De momento no es necesario

In [6]:
#!python -m spacy download en_core_web_sm

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")
df_guide.shape

(649, 6)

Guardo todo en una variable

In [6]:

text_list =[]

for i,row in df_guide.iterrows():
    #level0 = row.loc['level0'].replace('¶','')
    #level1 = row.loc['level1'].replace('¶','')
    #level2 = row.loc['level2'].replace('¶','')
    #level3 = row.loc['level3'].replace('¶','')
    #level4 = row.loc['level4'].replace('¶','')
    content= row.loc['content'].replace('¶','')
    text_list.append(content)
  
text = '. '.join(text_list)


# tratamiento de texto con tensor flow y keras

## Tokenización

Voy a tokenizar el texto a nivel carácter:

In [7]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)

word_vector =tokenizer.texts_to_sequences(['neighbor'])
print(word_vector)
vector_word=tokenizer.sequences_to_texts(word_vector)
print(vector_word)


nDstinctChar = len(tokenizer.word_index)
nChars = tokenizer.document_count
print(nDstinctChar,nChars)


[[8, 2, 5, 19, 11, 20, 7, 9]]
['n e i g h b o r']
79 550102


## dividir el conjunto de datos secuencialmente

El conjunto de entrenamiento sera el 90%, 5% para el de validadción y 5% para test

In [8]:
[codificacion] =np.array(tokenizer.texts_to_sequences([text]))-1 # la codificación irá de 0 a 78
train_size =nChars * 90 //100
train_set = tf.data.Dataset.from_tensor_slices(codificacion[:train_size])
len(train_set)

495091

train_set es un vector con casi 600000 elementos, para pasarlo a una red neuronal necesito dividirlo en pequeñas porciones de texto, de unos 100 caracteres por ejemplo. El método window permite realizar esto. Comenzará en a crear ventanas de 100 elementos desde la posición uno, pasando a la dos, tres..., creando un conjuto de vectores de 100 elementos de (600000 / 100) * 600000 aproximadamente.

la ventana se configura con el tramaño, shift es el número de pasos que avanza la ventana cada vez y drop_remainder a True hará que el tamaño de la venatana no vaya disminuyendo

In [10]:
window_size = 100 + 1 # es el tamaño del subvector =100 mas el paso
train_set=train_set.repeat().window(window_size,shift=1,drop_remainder=True)


aplanamos el conjunto de datos con el tamaño de la ventana (101)

In [57]:
train_set =train_set.flat_map(lambda window : window.batch(window_size))

Se realiza un mezclado de las ventanas

In [58]:
batch_size = 32
train_set = train_set.shuffle(10000).batch(batch_size)
train_set = train_set.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Voy a realizar la codificación one-hot para crear la bolsa de palabras con los 79 caracteres distintos que se manejaban, y añado la precarga

In [59]:
train_set = train_set.map(lambda X, y:(tf.one_hot(X, depth=nDstinctChar),y))
train_set=train_set.prefetch(1)

In [60]:
for X_batch, Y_batch in train_set.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 79) (32, 100)


## Creación de la red neuronal

In [62]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, nDstinctChar],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(nDstinctChar,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(train_set, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Epoch 1/10
15471/15471 [==============================] - 5579s 360ms/step - loss: 1.6821
Epoch 2/10
15471/15471 [==============================] - 2444s 158ms/step - loss: 1.3279
Epoch 3/10
15471/15471 [==============================] - 2447s 158ms/step - loss: 1.2821
Epoch 4/10
15471/15471 [==============================] - 2456s 159ms/step - loss: 1.2589
Epoch 5/10
15471/15471 [==============================] - 2448s 158ms/step - loss: 1.2451
Epoch 6/10
15471/15471 [==============================] - 2448s 158ms/step - loss: 1.2362
Epoch 7/10
15471/15471 [==============================] - 2446s 158ms/step - loss: 1.2280
Epoch 8/10
15471/15471 [==============================] - 2444s 158ms/step - loss: 1.2218
Epoch 9/10
15471/15471 [==============================] - 2446s 158ms/step - loss: 1.2177
Epoch 10/10
15471/15471 [==============================] - 2447s 158ms/step - loss: 1.2135


## Preprocesamiento entrada

Para probar el modelo voy a crear unas funciones auxiliares que realicen el posprocesamiento: la tokenización y la codificación one-hot.
Creo una función para predecir el siguiente caracter y otra para que cree un bucle y genere texto

In [110]:

def treament(input_list):
    input_token =np.array(tokenizer.texts_to_sequences(input_list))-1
    return tf.one_hot(input_token,nDstinctChar)

def next_char(input):
    aux = treament([input])
    predic = np.argmax(model(aux),axis=-1)
    #predic = model(input)[0,-1:,:].numpy() +1
    return tokenizer.sequences_to_texts(predic+1)[0][-1]



def complete_text(text, n_chars=80, temperature=1):
    for i in range(n_chars):
        text += next_char(text)
    return text

input = 'support vector'
complete_text(input)

'support vector machines (e.g. the coefficients \\(\\ell_2\\) is the coefficients \\(\\ell_2\\) is th'

Parece que el modelo, a la hora de predecir, se queda enganchado y repite la misma frase una y otra vez, es más pong lo que pongo siempre converge en el mismo discurso. Necesita introducir algo de aletoriedad

In [113]:
def next_char(input):
    aux = treament([input])
    char_prob = model.predict(aux)[0, -1:,:]
    rescaled_prob = tf.math.log(char_prob) / 0.8
    char_categ = tf.random.categorical(rescaled_prob,num_samples=1)
    return tokenizer.sequences_to_texts(char_categ.numpy() +1 )[0]

input = 'Linear model'
complete_text(input)

'Linear model that computes the above similarity is the\ncoefficients \\(\\ell_0\\) is the model '

## Guardar el modelo de generciónde texto por caracteres

In [117]:
# serializar el modelo a JSON
model_json = model.to_json()
with open("char_model/model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("char_model/model.h5")


Mejorando el modelo anterior a través del estado

In [11]:

#Creo el dataset de train con el 90%
train_ds_estado = tf.data.Dataset.from_tensor_slices(codificacion[:nChars * 90 // 100])
ds_size=len(train_ds_estado)
#creo las ventanas, esta vez no hay ni solapamiento, el númreo de ventanas se reduce
#tampoco hay mezcla, ya que las ventanas tienen que ser secuenciales donde acaba 1 empieza otra
train_ds_estado = train_ds_estado.window(window_size,shift =100, drop_remainder=True)
train_ds_estado=train_ds_estado.flat_map(lambda w: w.batch(window_size))
#los lotes serán equivalentes a la ventana, de esta manera se respetará la secuencialida entre lotes
train_ds_estado =train_ds_estado.batch(1)
train_ds_estado = train_ds_estado.map(lambda w: (w[:,:-1],w[:,1:]))
train_ds_estado =train_ds_estado.map(lambda X,y: (tf.one_hot(X, depth=nDstinctChar),y))
train_ds_estado=train_ds_estado.prefetch(1)
ds_size

495091

In [14]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape= [1, None,nDstinctChar]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(nDstinctChar, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])


In [16]:
steps_per_epoch = ds_size // 100
history = model.fit(train_ds_estado, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
4950/4950 [==============================] - 241s 49ms/step - loss: 1.4999 - accuracy: 0.5570
Epoch 2/50
4950/4950 [==============================] - 236s 48ms/step - loss: 1.4988 - accuracy: 0.5576
Epoch 3/50
4950/4950 [==============================] - 233s 47ms/step - loss: 1.4989 - accuracy: 0.5578
Epoch 4/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4967 - accuracy: 0.5582
Epoch 5/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4967 - accuracy: 0.5581
Epoch 6/50
4950/4950 [==============================] - 233s 47ms/step - loss: 1.4977 - accuracy: 0.5577
Epoch 7/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4932 - accuracy: 0.5595
Epoch 8/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4927 - accuracy: 0.5590
Epoch 9/50
4950/4950 [==============================] - 236s 48ms/step - loss: 1.4929 - accuracy: 0.5592
Epoch 10/50
4950/4950 [==============================] 

El poblema del modelo anterior es que sólo permite hacer predicciones para lotes del mismo tamaño que el entrenado, por lo que la entrada debes de ser de 100 carácteres. Para poder hacer que la entrada tenga un tamaño sin determinar hay que crear una red neuranoal sin estado igual, y copiar los pesos del anterior modelo con estado entrenado a este nuevo modelo

In [13]:
base_model_statless = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, nDstinctChar]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(nDstinctChar,activation="softmax"))
])

Para preparar el anterior modelo para que puedad guardar los pesos del modelo con estado habrá que especificar la estructura del tensor utilizado antes, que era la siguiente (\[1, None,nDstinctChar\]). Para permitir una entrada con cualquier tamaño:

In [160]:
base_model_statless.build(tf.TensorShape([None,None,nDstinctChar]))
base_model_statless.set_weights(model.get_weights())

In [ ]:
aprovecahndo el código anterior

In [163]:
def treament(input_list):
    input_token =np.array(tokenizer.texts_to_sequences(input_list))-1
    return tf.one_hot(input_token,nDstinctChar)

def next_char(input,model):
    aux = treament([input])
    char_prob = model.predict(aux)[0, -1:,:]
    rescaled_prob = tf.math.log(char_prob) / 1
    char_categ = tf.random.categorical(rescaled_prob,num_samples=1)
    return tokenizer.sequences_to_texts(char_categ.numpy() +1 )[0]

def complete_text(text, model,n_chars=80, temperature=1):
    for i in range(n_chars):
        text += next_char(text,model)
    return text

input = 'line'
complete_text(input, base_model_statless)

'support vectors, results. with dimensionality requires a list. the cost of dorical feature ind'

In [17]:
# serializar el modelo a JSON
model_json = model.to_json()
with open("char_model_state/model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("char_model_state/model.h5")